# Preprocessamento

O Preprocessamento dos dados ocorre após a etapa de Análise Exploratória, na qual algumas transformações foram aplicadas à matriz de dados. Assim, a matriz carregada neste momento não corresponde aos dados brutos, mas sim à versão modificada após o EDA.

# Importação dos dados

In [ ]:
from google.colab import files

# Abre a janela para selecionar o arquivo
uploaded = files.upload()

# Depois, para ler o arquivo
import pandas as pd

matriz = pd.read_csv('matriz_EDA.csv')

# 1. Multicolinearidade - análise antes do escalonamento

## 1.1. Correlação de Pearson/Spearman - valores

Correlação de Pearson/Spearman - valores: a correlação de Pearson é usada para medir a relação linear entre duas variáveis numéricas, enquanto a Spearman mede a relação monotônica. Ambas fornecem valores numéricos que indicam a força e a direção da relação entre variáveis, como entre MonthlyCharges e TotalCharges.

In [ ]:
# Correlação de Pearson/Spearman - valores
# Pearson: avalia relação linear (boa para variáveis contínuas normalmente distribuídas).
# Spearman: avalia relação monótona (serve também para não lineares ou ordinais).
# Correlação alta entre variáveis independentes pode causar multicolinearidade em modelos preditivos (e aí entra o VIF).

# Correlação	Valor	Interpretação
# Positiva forte	+0.7 a +1.0	Ambas crescem juntas
# Positiva moderada	+0.4 a +0.69	Tendência de crescer juntas
# Fraca / quase nula	-0.3 a +0.3	Pouca ou nenhuma relação linear
# Negativa moderada	-0.4 a -0.69	Uma sobe enquanto a outra desce
# Negativa forte	-0.7 a -1.0	Relação inversa forte


# Seleciona apenas as colunas numéricas
numericas = matriz.select_dtypes(include='number')

# Correlação de Pearson
print("\nCorrelação de Pearson:")
print(numericas.corr(method='pearson'))

# Correlação de Spearman
print("\nCorrelação de Spearman:")
print(numericas.corr(method='spearman'))

## 1.2. Correlação de Pearson/Spearman - heatmap

Correlação de Pearson/Spearman - heatmap: exibe visualmente as correlações entre múltiplas variáveis, usando uma matriz de correlação com cores para indicar a intensidade da relação. Essa análise é útil para identificar rapidamente quais variáveis têm correlação forte ou fraca entre si, facilitando a compreensão das interações entre os dados.

In [ ]:
# Correlação de Pearson/Spearman -heatmap
import seaborn as sns
import matplotlib.pyplot as plt

# Seleciona colunas numéricas
numericas = matriz.select_dtypes(include='number')

# Heatmap da correlação de Pearson
plt.figure(figsize=(20, 14))
sns.heatmap(numericas.corr(method='pearson'), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Mapa de Correlação - Pearson")
plt.show()

# Heatmap da correlação de Spearman
plt.figure(figsize=(20, 14))
sns.heatmap(numericas.corr(method='spearman'), annot=True, cmap='YlGnBu', fmt=".2f")
plt.title("Mapa de Correlação - Spearman")
plt.show()

## 1.3. Análise de variáveis altamente correlacionadas

O código realiza uma análise de variáveis altamente correlacionadas no conjunto de dados. A ideia é identificar duplas de variáveis que possuem uma forte correlação, o que pode ser problemático em modelos de aprendizado de máquina ou regressão, pois pode levar a multicolinearidade e dificultar a interpretação dos resultados.

In [ ]:
#Análise de variáveis altamente correlacionadas
import numpy as np

# Threshold de correlação alta
threshold = 0.85

# Seleciona apenas colunas numéricas
numericas = matriz.select_dtypes(include='number')

# Matriz de correlação absoluta
cor_matrix = numericas.corr().abs()

# Pega a parte superior da matriz para evitar duplicações
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))

# Identifica colunas com correlação acima do limiar
altamente_correlacionadas = [col for col in upper_tri.columns if any(upper_tri[col] > threshold)]

print(f"\nVariáveis altamente correlacionadas (r > {threshold}):")
print(altamente_correlacionadas)


## 1.4. Detecção de relações não-lineares

O pairplot ajuda a visualizar relações não-lineares entre as variáveis numéricas, o que é importante porque a correlação de Pearson (usada nas análises lineares) não captura relações não-lineares. Ou seja, ele permite perceber padrões não-lineares ou interações complexas entre as variáveis que podem ser importantes para a modelagem.

* Identificação de padrões não-lineares: Variáveis que não possuem uma correlação linear forte, mas que podem estar relacionadas de outras formas, como curvas ou outras formas não-lineares.

* Insights sobre os dados: Pode ajudar a identificar transformações de variáveis ou interações entre elas que podem ser úteis em modelos de aprendizado de máquina, como a criação de novas características ou o uso de técnicas específicas para capturar relações não-lineares (por exemplo, árvores de decisão).

In [ ]:
# Detecção de Relações Não-Lineares (via Pairplot + Spearman)
# Pairplot ajuda a visualizar relações não-lineares
sns.pairplot(matriz.select_dtypes(include='number'), kind='scatter', corner=True)
plt.suptitle("Pairplot para relações não-lineares", y=1.02)
plt.show()

## 1.5. Boxplots por classe do target

O código realiza a análise de variáveis numéricas em relação à variável alvo, 'Churn' (que provavelmente representa se o cliente abandonou ou não o serviço), utilizando boxplots para verificar a distribuição de cada variável numérica para cada classe do target (Churn).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Verifica se a variável 'Churn' está no DataFrame
if 'Churn' in matriz.columns:
    # Selecionar variáveis numéricas (excluindo o target se for numérico)
    variaveis_numericas = matriz.select_dtypes(include='number').columns.drop('Churn', errors='ignore')

    for var in variaveis_numericas:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='Churn', y=var, data=matriz)
        plt.title(f"Boxplot de {var} por classe do target (Churn)")
        plt.tight_layout()
        plt.show()
else:
    print("A variável 'Churn' não foi encontrada no DataFrame.")

## 1.6. Correlação com o target

O código calcula a correlação de Pearson entre a variável target ('Churn') e as variáveis numéricas do DataFrame, para identificar o grau de relacionamento linear entre cada variável numérica e o target.

A correlação de Pearson mede o grau de relacionamento linear entre duas variáveis. Ela varia entre -1 e 1, onde:

* 1 indica uma correlação positiva perfeita (à medida que uma variável aumenta, a outra também aumenta).
* -1 indica uma correlação negativa perfeita (à medida que uma variável aumenta, a outra diminui).
* 0 indica nenhuma correlação linear.

In [ ]:
# Correlação com o target (somente variáveis numéricas)
# Correlação de Pearson entre variáveis numéricas e o target
correlacoes = matriz.corr(numeric_only=True)['Churn'].drop('Churn').sort_values(ascending=False)
print("Correlação com o target:")
print(correlacoes)



# 2. Separação do dataset - treinamento e teste

In [ ]:
# Separação dos dados em treinamento e teste
#matriz (completa)
#dados_train (treino)
#dados_test (teste)
# 80% treinamento e 20% teste
# random_state para garantir a reprodutibilidade
from sklearn.model_selection import train_test_split

# Separação
dados_train, dados_test = train_test_split(matriz,
                                           test_size=0.2,
                                           random_state=35)

# Conferência
print("Matriz completa: {} instâncias\n{} instâncias de treino\n{} instâncias de teste".
      format(len(matriz), len(dados_train), len(dados_test)))


# 3. Escalonamento - zscore

In [ ]:
#Escalonamento - z-score
#Escalonar no treino e guardar a média e o desvio padrão para aplicar nos dados de teste.
# O StandardScaler aplica o Z-Score em cada variável, mas como é preciso guardar média e desvio, foi feito de maneira manual.
# Z-score: subtrai a média (centraliza os dados em 0) e divide pelo desvio padrão (ajusta a escala para variância 1)
# o StandardScaler, na prática, está fazendo o Z-Score em todas as colunas numéricas escolhidas
import pandas as pd
import numpy as np

# Seleciona apenas colunas numéricas (excluindo a coluna 'Churn')
colunas_numericas = dados_train.drop('Churn', axis=1).select_dtypes(include=np.number).columns

# Calcula a média e o desvio padrão apenas com os dados de treino
medias = dados_train[colunas_numericas].mean()
desvios = dados_train[colunas_numericas].std()

# Salva essas informações
media_desvio_dict = {
    'medias': medias,
    'desvios': desvios
}

# Faz o escalonamento do treino
x_dados_train_escalado = dados_train.copy()
x_dados_train_escalado[colunas_numericas] = (dados_train[colunas_numericas] - medias) / desvios

# Faz o escalonamento do teste usando as médias e desvios do treino
x_dados_test_escalado = dados_test.copy()
x_dados_test_escalado[colunas_numericas] = (dados_test[colunas_numericas] - medias) / desvios

# Cria as variáveis de target (y) para treino e teste
y_dados_train_escalado = dados_train['Churn']
y_dados_test_escalado = dados_test['Churn']

In [ ]:
x_train= x_dados_train_escalado.drop('Churn', axis=1)
x_test= x_dados_test_escalado.drop('Churn', axis=1)
y_train=y_dados_train_escalado
y_test=y_dados_test_escalado

# 4. Multicolinearidade - Análise após o escalonamento

In [ ]:
matriz=x_train

## 4.1. VIF (Variance Inflation Factor) para multicolinearidade

O VIF (Variance Inflation Factor) é uma métrica utilizada para verificar a multicolinearidade em um conjunto de dados, ou seja, a correlação entre variáveis independentes. Quando variáveis independentes estão altamente correlacionadas entre si, a multicolinearidade pode distorcer as estimativas de um modelo de regressão, tornando-o menos confiável.

* O VIF mede o quanto a variabilidade de uma variável independente está inflacionada devido à correlação com outras variáveis independentes.

* Um VIF alto (geralmente maior que 5 ou 10, dependendo da referência) indica multicolinearidade alta, o que pode afetar a precisão dos coeficientes do model

In [ ]:
# VIF (Variance Inflation Factor) para multicolinearidade
# Valores de VIF > 5 (ou 10, a depender da referência) indicam multicolinearidade alta.
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

# Seleciona variáveis numéricas
X = matriz.select_dtypes(include='number').dropna()  # Remove linhas com NaN

# Adiciona constante para o cálculo do VIF
X_const = add_constant(X)

# Calcula VIF
vif = pd.DataFrame()
vif['Variável'] = X.columns
vif['VIF'] = [variance_inflation_factor(X_const.values, i + 1) for i in range(len(X.columns))]

print("\nVIF (Variance Inflation Factor):")
print(vif.sort_values(by='VIF', ascending=False))


## 4.2. Matriz de correlação

In [ ]:
# matriz de correlação
#Quando duas variáveis têm correlação muito alta (ex: > 0,9 ou < -0,9), pode ser problema.

import seaborn as sns
import matplotlib.pyplot as plt

# Calcular a matriz de correlação
correlacao = matriz.corr(numeric_only=True)

# Visualizar
plt.figure(figsize=(12, 8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

# Como resolver a multicolinearidade:
  #Remover uma das variáveis altamente correlacionadas.
  #Combinar variáveis correlacionadas (ex: média, soma).
  #Reduzir Dimensionalidade: usar técnicas como PCA (Principal Component Analysis).
  #Modelos robustos: alguns algoritmos lidam bem com multicolinearidade (ex: árvore de decisão).


## 4.3. Determinante da matriz de correlação

In [ ]:
# Determinante da matriz de correlação
# Se o determinante for próximo de 0 → existe multicolinearidade.
import numpy as np

corr_matrix = matriz.corr(numeric_only=True)
determinante = np.linalg.det(corr_matrix)

print(f"Determinante da matriz de correlação: {determinante:.6f}")


## 4.4. Eigenvalues (autovalores) da matriz de correlação

In [ ]:
#Eigenvalues (autovalores) da matriz de correlação
#Quando um ou mais autovalores são muito pequenos (próximos de zero), indica que há combinações lineares entre variáveis.
eigenvalues, _ = np.linalg.eig(corr_matrix)

print("Autovalores:")
print(eigenvalues)


# 5. Solução para multicolinearidade - PCA

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

# 1. Aplicação do PCA com 95% da variância explicada
pca = PCA(n_components=0.95)
x_train_pca = pca.fit_transform(x_train) # PCA ajustado com base no treino
x_test_pca = pca.transform(x_test) # PCA aplica a transformação com base no ajuste anterior

# 2. Exibição das informações solicitadas
print("Número de colunas antes do PCA:", x_train.shape[1])
print("Número de componentes principais após o PCA:", pca.n_components_)
print("Variância explicada por componente:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.4f}")

# 3. Conversão para DataFrame com nomes de colunas "PC1", "PC2", ...
pc_columns = [f"PC{i+1}" for i in range(pca.n_components_)]
x_train_pca_df = pd.DataFrame(x_train_pca, columns=pc_columns)
x_test_pca_df = pd.DataFrame(x_test_pca, columns=pc_columns)

# Exibe as primeiras linhas
print("\nPrimeiras linhas do x_train_pca:")
print(x_train_pca_df.head())

In [ ]:
x_train=x_train_pca_df
x_test=x_test_pca_df

# 6. Salvar as matrizes de treinamento e teste

In [ ]:
from google.colab import files

# Treinamento
# Salvar o DataFrame de treino como CSV
x_train.to_csv('x_train.csv', index=False)
# Fazer o download do arquivo CSV
files.download('x_train.csv')

# Salvar o target de treino como CSV
y_train.to_csv('y_train.csv', index=False)
# Fazer o download do arquivo CSV
files.download('y_train.csv')


# Teste
# Salvar o DataFrame de teste como CSV
x_test.to_csv('x_test.csv', index=False)
# Fazer o download do arquivo CSV
files.download('x_test.csv')
# Salvar o target de teste como CSV
y_test.to_csv('y_test.csv', index=False)
# Fazer o download do arquivo CSV
files.download('y_test.csv')